In [29]:
from flask import Flask, request, jsonify
import os
import serpapi
import pandas as pd
import tensorflow as tf
from keras.preprocessing import image
import numpy as np


In [41]:
import json
from flask import Flask

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import os
import logging


app = Flask(__name__)

logging.basicConfig(level=logging.INFO)
# Load dataset
train_path = "C:/Users/rashe/ImageRecognitionProject/Groceries-Images-Recognition/ML/dataset/train"
image_category = os.listdir(train_path)

# Apply the suitable Image Preprocessing by using ImageDataGenerator
train_generator = ImageDataGenerator(rescale=1.0/255.0)
train_image_generator = train_generator.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


# Load the trained model
model = tf.keras.models.load_model('C:/Users/rashe/ImageRecognitionProject/Groceries-Images-Recognition/ML/mymodel.h5')

# Define the class map for label mapping
class_map = dict([v, k] for k, v in train_image_generator.class_indices.items())

@app.route('/')
def index():
    return 'Image Recognition API'

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Get the image from the POST request
        img_file = request.files['image']
        
        # Load and preprocess the image
        img = load_img(img_file, target_size=(224, 224))
        img_array = img_to_array(img) / 255.0
        img_input = img_array.reshape((1, img_array.shape[0], img_array.shape[1], img_array.shape[2]))

        # Make prediction
        predicted_label = int(model.predict(img_input).argmax(axis=-1))
        predicted_img = class_map[predicted_label]

        # Return the result
        return jsonify({"predicted_product": predicted_img})

    except Exception as e:
        logging.error(f"Error during prediction: {e}")
        return jsonify({"error": "Failed to predict"}), 500

if __name__ == '__main__':
    app.run(debug=False)

Found 3115 images belonging to 36 classes.


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
INFO:werkzeug:Press CTRL+C to quit
127.0.0.1 - - [15/Apr/2024 02:05:36] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2024 02:05:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2024 02:05:36] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2024 02:05:36] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [15/Apr/2024 02:05:57] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2024 02:05:57] "GET / HTTP/1.1" 200 -
